# Types of Agent Architectures

1. Single Multi-agent (Sequential flow)

2. Supervisor Multi-agent (Orchestrator based - supervisor delegates the tasks)
3. Hierarchial Architecture

## 1. Single Multi-agent Architecture

Researcher and Writer Agents

### Importing required libraries

In [6]:
import os
from typing import TypedDict, Annotated, List, Literal

# For LangChain
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

# For LangGraph
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

### Initializing Groq LLM

In [5]:
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(model="llama-3.1-8b-instant")

llm.invoke("Hello")

AIMessage(content='Hello, how are you today? Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 36, 'total_tokens': 59, 'completion_time': 0.030666667, 'prompt_time': 0.002014084, 'queue_time': 0.051407867, 'total_time': 0.032680751}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8ab2e50475', 'finish_reason': 'stop', 'logprobs': None}, id='run--3e80f8bd-8389-4e66-b155-41e227eca14d-0', usage_metadata={'input_tokens': 36, 'output_tokens': 23, 'total_tokens': 59})

### Creating State

In [11]:
def AgentState(MessagesState):
    next_agent: str # The next agent to invoke

### Creating TavilySearch Tool for `Researcher Agent`

In [8]:
@tool
def tavily_search_tool(query: str) -> str:
    """ Search the web for latest information.  """
    search = TavilySearchResults(max_results=3)
    results = search.invoke(query)

    return str(results)

### Creating WriteSummary Tool for `Content Writer Agent`

In [9]:
@tool
def write_summary(content: str) -> str: 
    """ Write a summary of the provided content. """

    summary = f"Summary of findings:\n\n{content[:500]}"
    return summary

### Defining `Researcher Agent` Node

In [12]:
def researcher_agent(agentState: AgentState): 

    # fetching the messages from agentState
    messages = agentState["messages"]

    # creating the system message for the agent
    system_message = SystemMessage(content="You are a research assistant. Use the search_web tool to find information about the user's request.")

    # Binding tavily search tool with the llm
    llm_with_tool = llm.bind_tools([tavily_search_tool])

    # Invoking the LLM with the system message and user messages
    response = llm_with_tool.invoke([system_message] + messages)

    # Returning the response and routing to the next agent i.e., Content Writer
    return {
        "messages": [response],
        "next_agent": "writer"
    }

### Defining `Writer Agent` Node

In [13]:
def writer_agent(agentState: AgentState): 

    # fetching the messages from agentState
    messages = agentState["messages"]

    # creating the system message for the agent
    system_message = SystemMessage(content="You are a technical writer. Use the write_summary tool, review the conversation and create a clear, concise summary of the findings.")

    # Binding tavily search tool with the llm
    llm_with_tool = llm.bind_tools([write_summary])

    # Invoking the LLM with the system message and user messages
    response = llm_with_tool.invoke([system_message] + messages)

    # Returning the response and routing to the next agent i.e., Content Writer
    return {
        "messages": [response],
        "next_agent": "end"
    }